<a href="https://colab.research.google.com/github/yala/deeplearning_bootcamp/blob/master/lab4/advanced_vision_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Computer Vision Exercise: CIFAR 10


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())


!pip install torch torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [0]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np


# The Task: CIFAR10
<img src="https://corochann.com/wp-content/uploads/2017/04/cifar10_plot.png">

## Step 1: Loading Data and Preprocessing
Let's start by loading the data.
We're going to normalize our images to have 0 mean, and unit variance. We'll do this using some [torchvision](https://pytorch.org/docs/stable/torchvision/index.html) transforms. This generally helps stablize learning, and is common practice. 

In [0]:
# Img mean value of .13, and stdv of .31 were computed across entire train set
# in prior work
normalize_image = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.49,.48,.44), (0.25,.24,.26))
                ])

# Dataset is loaded fro torchvision
all_train = datasets.CIFAR10('data', train=True, download=True, transform=normalize_image)

num_train = int(len(all_train)*.8)
train = [all_train[i] for i in range(num_train)]
dev = [all_train[i] for i in range(num_train,len(all_train))]
test = datasets.CIFAR10('data', train=False, download=True, 
                      transform=normalize_image)
                           


In [0]:
train[0][0].size()


In [0]:
def train_epoch( model, train_loader, optimizer, epoch):
    model.train() # Set the nn.Module to train mode. 
    model = model.to('cuda')
    total_loss = 0
    correct = 0
    num_samples = len(train_loader.dataset)
    for batch_idx, (x, target) in enumerate(train_loader): #1) get batch
        x, target = x.to('cuda'), target.to('cuda')
        B, C, H, W = x.size()
        x = x.expand([B,3,H,W]).contiguous()
        # Reset gradient data to 0
        optimizer.zero_grad()
        # Get prediction for batch
        output = model(x)
        # 2) Compute loss
        loss = F.cross_entropy(output, target)
        #3) Do backprop
        loss.backward()
        #4) Update model
        optimizer.step()
        
        ## Do book-keeping to track accuracy and avg loss
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        total_loss += loss.detach() # Don't keep computation graph 

    print('Train Epoch: {} \tLoss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            epoch, total_loss / num_samples, 
            correct, 
            num_samples,
            100. * correct / num_samples))


In [0]:
def eval_epoch(model, test_loader, name):
    model = model.to('cuda')
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to('cuda'), target.to('cuda')
        B, C, H, W = data.size()
        data = data.expand([B,3,H,W]).contiguous()
        output = model(data)
        test_loss += F.cross_entropy(output, target).item() # sum up batch loss
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name,
        test_loss, 
        correct, 
        len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


## Step 2: Building a model

All pytorch models should be implemented as instances of `nn.Module`. 

Try your own custom model or use a pretrained one from torchvision

In [0]:
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(512,10)

In [0]:
# Training settings
epochs = 10
lr = 1e-3

batch_size = 64
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
dev_loader = torch.utils.data.DataLoader(dev, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)


optimizer = optim.Adam(model.parameters(), lr=lr)


In [0]:

for epoch in range(1, epochs + 1):
    train_epoch(model, train_loader, optimizer, epoch)
    eval_epoch(model,  dev_loader, "Dev")
    print("---")

In [0]:
eval_epoch(model,  test_loader, "Test")